<a href="https://colab.research.google.com/github/SindhujaPandian/MachineLearningProjects/blob/master/Diabetes_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip install pyspark==2.4.4

     |████████████████████████████████| 215.7MB 61kB/s 
     |████████████████████████████████| 204kB 36.8MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130388 sha256=b7ab03f7bf568c1074afcba4d30b3704fe08edab910bc0322a3ebcf10bb3fed1
  Stored in directory: /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark


In [ ]:
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("spark").getOrCreate()

In [7]:
! git clone https://github.com/education454/diabetes_dataset

Cloning into 'diabetes_dataset'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.


In [8]:
! ls diabetes_dataset

diabetes.csv


In [9]:
df = spark.read.csv('/content/diabetes_dataset/diabetes.csv',header=True, inferSchema=True)

In [10]:
df.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          2|    138|           62|           35|      0|33.6|                   0.127| 47|      1|
|          0|     84|           82|           31|    125|38.2|                   0.233| 23|      0|
|          0|    145|            0|            0|      0|44.2|                    0.63| 31|      1|
|          0|    135|           68|           42|    250|42.3|                   0.365| 24|      1|
|          1|    139|           62|           41|    480|40.7|                   0.536| 21|      0|
|          0|    173|           78|           32|    265|46.5|                   1.159| 58|      0|
|          4|     99|           72|           17|      0|25.6|                   0.294| 28|      0|


In [11]:
df.printSchema()

root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nullable = true)
 |-- BloodPressure: integer (nullable = true)
 |-- SkinThickness: integer (nullable = true)
 |-- Insulin: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Outcome: integer (nullable = true)



In [12]:
print(df.count(),len(df.columns))

2000 9


In [13]:
df.groupby('outcome').count().show()

+-------+-----+
|outcome|count|
+-------+-----+
|      1|  684|
|      0| 1316|
+-------+-----+



In [14]:
df.describe().show()

+-------+-----------------+------------------+------------------+-----------------+-----------------+------------------+------------------------+------------------+------------------+
|summary|      Pregnancies|           Glucose|     BloodPressure|    SkinThickness|          Insulin|               BMI|DiabetesPedigreeFunction|               Age|           Outcome|
+-------+-----------------+------------------+------------------+-----------------+-----------------+------------------+------------------------+------------------+------------------+
|  count|             2000|              2000|              2000|             2000|             2000|              2000|                    2000|              2000|              2000|
|   mean|           3.7035|          121.1825|           69.1455|           20.935|           80.254|32.192999999999984|     0.47092999999999974|           33.0905|             0.342|
| stddev|3.306063032730656|32.068635649902916|19.188314815604098|16.103242909926

In [16]:
#find minssing values
for col in df.columns:
  print(col+':',df[df[col].isNull()].count())


Pregnancies: 0
Glucose: 0
BloodPressure: 0
SkinThickness: 0
Insulin: 0
BMI: 0
DiabetesPedigreeFunction: 0
Age: 0
Outcome: 0


In [17]:
def count_zeros():
  column_list = ['Glucose','BloodPressure','SkinThickness','Insulin','BMI']
  for i in column_list:
    print(i+":",df[df[i]==0].count())

In [18]:
count_zeros()

Glucose: 13
BloodPressure: 90
SkinThickness: 573
Insulin: 956
BMI: 28


In [20]:
from pyspark.sql.functions import *
for i in df.columns[1:6]:
  data = df.agg({i:'mean'}).first()[0]
  print('Mean value for {} is {}'.format(i,int(data)))
  df = df.withColumn(i,when(df[i]==0,int(data)).otherwise(df[i]))

Mean value for Glucose is 121
Mean value for BloodPressure is 69
Mean value for SkinThickness is 20
Mean value for Insulin is 80
Mean value for BMI is 32


In [21]:
df.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          2|    138|           62|           35|     80|33.6|                   0.127| 47|      1|
|          0|     84|           82|           31|    125|38.2|                   0.233| 23|      0|
|          0|    145|           69|           20|     80|44.2|                    0.63| 31|      1|
|          0|    135|           68|           42|    250|42.3|                   0.365| 24|      1|
|          1|    139|           62|           41|    480|40.7|                   0.536| 21|      0|
|          0|    173|           78|           32|    265|46.5|                   1.159| 58|      0|
|          4|     99|           72|           17|     80|25.6|                   0.294| 28|      0|


In [24]:
for col in df.columns:
  print('Correlation to Outcome variable for {} is {}'.format(col,df.stat.corr('Outcome',col)))

Correlation to Outcome variable for Pregnancies is 0.22443699263363961
Correlation to Outcome variable for Glucose is 0.48796646527321064
Correlation to Outcome variable for BloodPressure is 0.17171333286446713
Correlation to Outcome variable for SkinThickness is 0.1659010662889893
Correlation to Outcome variable for Insulin is 0.1711763270226193
Correlation to Outcome variable for BMI is 0.2827927569760082
Correlation to Outcome variable for DiabetesPedigreeFunction is 0.1554590791569403
Correlation to Outcome variable for Age is 0.23650924717620253
Correlation to Outcome variable for Outcome is 1.0


In [31]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction','Age'],outputCol='features' )
output_data = assembler.transform(df)


In [32]:
output_data.printSchema()

root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nullable = true)
 |-- BloodPressure: integer (nullable = true)
 |-- SkinThickness: integer (nullable = true)
 |-- Insulin: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Outcome: integer (nullable = true)
 |-- features: vector (nullable = true)



In [33]:
output_data.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+--------------------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|            features|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+--------------------+
|          2|    138|           62|           35|     80|33.6|                   0.127| 47|      1|[2.0,138.0,62.0,3...|
|          0|     84|           82|           31|    125|38.2|                   0.233| 23|      0|[0.0,84.0,82.0,31...|
|          0|    145|           69|           20|     80|44.2|                    0.63| 31|      1|[0.0,145.0,69.0,2...|
|          0|    135|           68|           42|    250|42.3|                   0.365| 24|      1|[0.0,135.0,68.0,4...|
|          1|    139|           62|           41|    480|40.7|                   0.536| 21|      0|[1.0,139.0,62.0,4...|
|          0|    173|           

In [35]:
from pyspark.ml.classification import LogisticRegression
final_data = output_data.select('features','Outcome')


In [36]:
final_data.printSchema()

root
 |-- features: vector (nullable = true)
 |-- Outcome: integer (nullable = true)



In [37]:
final_data.show()

+--------------------+-------+
|            features|Outcome|
+--------------------+-------+
|[2.0,138.0,62.0,3...|      1|
|[0.0,84.0,82.0,31...|      0|
|[0.0,145.0,69.0,2...|      1|
|[0.0,135.0,68.0,4...|      1|
|[1.0,139.0,62.0,4...|      0|
|[0.0,173.0,78.0,3...|      0|
|[4.0,99.0,72.0,17...|      0|
|[8.0,194.0,80.0,2...|      0|
|[2.0,83.0,65.0,28...|      0|
|[2.0,89.0,90.0,30...|      0|
|[4.0,99.0,68.0,38...|      0|
|[4.0,125.0,70.0,1...|      1|
|[3.0,80.0,69.0,20...|      0|
|[6.0,166.0,74.0,2...|      0|
|[5.0,110.0,68.0,2...|      0|
|[2.0,81.0,72.0,15...|      0|
|[7.0,195.0,70.0,3...|      1|
|[6.0,154.0,74.0,3...|      0|
|[2.0,117.0,90.0,1...|      0|
|[3.0,84.0,72.0,32...|      0|
+--------------------+-------+
only showing top 20 rows



In [38]:
train , test = final_data.randomSplit([0.7,0.3])
models = LogisticRegression(labelCol='Outcome')
model = models.fit(train)

In [39]:
summary = model.summary

In [40]:
summary.predictions.describe().show()

+-------+------------------+-------------------+
|summary|           Outcome|         prediction|
+-------+------------------+-------------------+
|  count|              1394|               1394|
|   mean|0.3400286944045911|0.25968436154949787|
| stddev|0.4738884669912586|  0.438618745893471|
|    min|               0.0|                0.0|
|    max|               1.0|                1.0|
+-------+------------------+-------------------+



In [43]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [44]:
predictions = model.evaluate(test)

In [47]:
predictions.predictions.show(20)

+--------------------+-------+--------------------+--------------------+----------+
|            features|Outcome|       rawPrediction|         probability|prediction|
+--------------------+-------+--------------------+--------------------+----------+
|[0.0,57.0,60.0,20...|      0|[3.83238612700253...|[0.97880124452292...|       0.0|
|[0.0,78.0,88.0,29...|      0|[2.84136955321653...|[0.94487084563013...|       0.0|
|[0.0,78.0,88.0,29...|      0|[2.84136955321653...|[0.94487084563013...|       0.0|
|[0.0,84.0,64.0,22...|      0|[2.57055580731769...|[0.92894239262858...|       0.0|
|[0.0,84.0,82.0,31...|      0|[2.80246016002280...|[0.94280862181998...|       0.0|
|[0.0,84.0,82.0,31...|      0|[2.80246016002280...|[0.94280862181998...|       0.0|
|[0.0,86.0,68.0,32...|      0|[2.84542148620469...|[0.94508153047274...|       0.0|
|[0.0,86.0,68.0,32...|      0|[2.84542148620469...|[0.94508153047274...|       0.0|
|[0.0,91.0,68.0,32...|      0|[2.31961273742509...|[0.91048838405972...|    

In [48]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol = 'rawPrediction' , labelCol='Outcome')
evaluator.evaluate(model.transform(test))

0.8459114959114958

In [49]:
model.save('final_model')

In [50]:
from pyspark.ml.classification import LogisticRegressionModel
model = LogisticRegressionModel.load('final_model')